# Experiment Tracking com MLflow

Este notebook demonstra como usar o MLflow para tracking de experimentos com nosso modelo mockado de recomendação.

In [1]:
import os
import sys
import pandas as pd
import mlflow

# Adiciona o diretório src ao PYTHONPATH
project_dir = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_dir not in sys.path:
    sys.path.append(project_dir)

from src.config import configure_mlflow, get_config
from src.recomendation_model.mocked_model import MockedRecommender, MLflowWrapper
from src.features.schemas import get_model_signature, create_mock_input_example
from src.predict.predict import predict_for_userId
from src.evaluation.utils import evaluate_model
from src.train.utils import load_train_data
from src.data.data_loader import get_evaluation_data

2025-02-26 20:15:25,833 - INFO - Ambiente: dev
2025-02-26 20:15:31,726 - INFO - Ambiente: dev


ModuleNotFoundError: No module named 'constants'

## 1. Configuração do MLflow

In [ ]:
# Configura o MLflow
configure_mlflow()

In [ ]:
model_params = {
    'threshold': 0.5,
    'top_k': 5
}

## 2. Treinamento e Tracking

In [ ]:
X_train, y_train = load_train_data()
evaluation_data = get_evaluation_data()

input_example = create_mock_input_example()


# Inicia um novo experimento
with mlflow.start_run() as run:
    # Cria e treina o modelo
    model = MockedRecommender(**model_params)
    model.train(X_train, y_train)
    
    # Loga parâmetros
    mlflow.log_params(model_params)
    
    # Loga métricas
    metrics = evaluate_model(model, evaluation_data)
    mlflow.log_metrics(metrics)
    
    # Salva o modelo usando o wrapper e a signature
    wrapper = MLflowWrapper(model)
    mlflow.pyfunc.log_model(
        artifact_path=get_config('MODEL_NAME'),
        python_model=wrapper,
        signature=get_model_signature(),
        input_example=input_example
    )
    
    # Guarda o run_id
    run_id = run.info.run_id


2025/02/09 19:14:45 INFO mlflow.pyfunc: Validating input example against model signature


🏃 View run skillful-wolf-835 at: http://localhost:5001/#/experiments/1/runs/9f03d1b328414fc2a92038155baaa17c
🧪 View experiment at: http://localhost:5001/#/experiments/1


## 3. Carregamento e Predição

In [ ]:
loaded_model = mlflow.pyfunc.load_model(f"runs:/{run_id}/{get_config('MODEL_NAME')}")

# Faz predições com o modelo carregado
predictions = loaded_model.predict(input_example)
print(f"Predictions: {predictions}")

# Teste com o modelo original também
original_predictions = model.predict(input_example)
print(f"Original predictions: {original_predictions}")

Predictions: [0.5]
Original predictions: [0.5]
